In [37]:
import os
import pandas as pd
import warnings
import functions as fc

In [38]:
#Ocultar advertencias/warnings mas no los borra
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

In [39]:
#Año evaluado
Añoeval = 2025

**Variables globales de las rutas necesarias**

In [40]:
#Rutas del archivo de la COOISPI
Ruta_COOISPI = "G:/.shortcut-targets-by-id/1rqpfbdZ6z51epFv6ZwognhckW7HqBMjN/SIRI_2024_INFORMES_SOBREPESO/DATOS/Datos_COOISPI_Mezclas.csv"                        # Ruta COOISPI en csv
#Ruta_COOISPI_2024="/content/drive/MyDrive/Consolidado COOISPI/2024/Z09.xlsx"                                                                                      # Ruta COOISPI en el 2024
#Ruta_COOISPI_2025="/content/drive/MyDrive/Consolidado COOISPI/2025/Z09.xlsx"  

#Rutas del archivo del EGE
Ruta_Archivo_EGE = "G:/.shortcut-targets-by-id/1rqpfbdZ6z51epFv6ZwognhckW7HqBMjN/SIRI_2024_INFORMES_SOBREPESO/DATOS/Datos_Mezclas_2021.csv"
#Ruta_Archivo_EGE = "/content/drive/MyDrive/SIRI_2024_INFORMES_SOBREPESO/DATOS/Datos_Mezclas_2021.csv"                                                             # En formato csv en el drive de Melanny Herrera
#Ruta_Archivo_EGE = "/content/drive/MyDrive/appsheet/Coolab/Documentos/Mezclas/DB Empaque de Mezclas 2021_Copia.xlsm"                                              # Ruta EGE pruebas Melanny Herrera
#Ruta_Archivo_EGE = "/content/drive/MyDrive/EGE Colcafé Medellín 2018 - 2024/Data historica/2021-2024/DB Empaque de Mezclas 2021.xlsm"                             # Ruta EGE en el drive de Jesse Mauricio Beltran

#Rutas del archivo de Novedades
Ruta_Novedades = "G:/.shortcut-targets-by-id/1rqpfbdZ6z51epFv6ZwognhckW7HqBMjN/SIRI_2024_INFORMES_SOBREPESO/INFORME_MEZCLAS_MEDELLIN/Mezclas_Novedades_V2.xlsx"
#Ruta_Novedades = "/content/drive/MyDrive/SIRI_2024_INFORMES_SOBREPESO/INFORME_MEZCLAS_MEDELLIN/Mezclas_Novedades_V2.xlsx"                                         # Ruta del archivo de Novedades en el drive de Melanny Herrera
#Ruta_Novedades = '/content/drive/MyDrive/Proyectos/2024/SIRI_2024_INFORMES_SOBREPESO/INFORME_MEZCLAS_MEDELLIN/Mezclas_Novedades.xlsx'                             # Ruta del archivo de Novedades en el drive de Jesse Mauricio Beltran

#Rutas del archivo de Consolidados
Ruta_Consolidados = "G:/.shortcut-targets-by-id/1rqpfbdZ6z51epFv6ZwognhckW7HqBMjN/SIRI_2024_INFORMES_SOBREPESO/INFORME_MEZCLAS_MEDELLIN/Consolidado_V2.xlsx"
#Ruta_Consolidados ="/content/drive/MyDrive/SIRI_2024_INFORMES_SOBREPESO/INFORME_MEZCLAS_MEDELLIN/Consolidado_V2.xlsx"                                             # Ruta del archivo de Consolidados en el drive de Melanny Herrera
#Ruta_Consolidados ='/content/drive/MyDrive/Proyectos/2024/SIRI_2024_INFORMES_SOBREPESO/INFORME_MEZCLAS_MEDELLIN/Consolidado.xlsx'                                 # Ruta del archivo de Novedades en el drive de Jesse Mauricio Beltran

#Ruta de salida
Ruta_Mezclas = "G:/.shortcut-targets-by-id/1rqpfbdZ6z51epFv6ZwognhckW7HqBMjN/SIRI_2024_INFORMES_SOBREPESO/INFORME_MEZCLAS_MEDELLIN/"
#Ruta_Mezclas = "/content/drive/MyDrive/SIRI_2024_INFORMES_SOBREPESO/INFORME_MEZCLAS_MEDELLIN/"                                                                    # Ruta del archivo consolidado csv en el drive de Melanny Herrera
#Ruta_Mezclas = "/content/drive/MyDrive/Proyectos/2024/SIRI_2024_INFORMES_SOBREPESO/INFORME_MEZCLAS_MEDELLIN/"                                                     # Ruta del archivo consolidado csv en el drive de Jesse Mauricio Beltran

#Validacion de existencia de las rutas
for nombre, ruta in {
  "Ruta_COOISPI": Ruta_COOISPI,
  "Ruta_Archivo_EGE": Ruta_Archivo_EGE,
  "Ruta_Novedades": Ruta_Novedades,
  "Ruta_Consolidados": Ruta_Consolidados,
  "Ruta_Mezclas": Ruta_Mezclas,
}.items():
  if not os.path.exists(ruta):
    raise FileNotFoundError(f"La ruta '{nombre}' no fue encontrada.")
  else:
    print(f"Ruta '{nombre}' encontrada.")

Ruta 'Ruta_COOISPI' encontrada.
Ruta 'Ruta_Archivo_EGE' encontrada.
Ruta 'Ruta_Novedades' encontrada.
Ruta 'Ruta_Consolidados' encontrada.
Ruta 'Ruta_Mezclas' encontrada.


In [41]:
#Leer_archivo COOISPI csv
df_COOISPI_COMB = fc.leer_archivo(Ruta_COOISPI)

Leyendo archivo...


In [42]:
#Convertir columna 'Codigo' a str para futuros procesor (merge y tabla pivote)
df_COOISPI_COMB['Codigo'] = df_COOISPI_COMB['Codigo'].astype(str)

In [43]:
#Funcion para leer archivo el EGE/TPM
df_Mezclas = fc.leer_archivo(Ruta_Archivo_EGE, 'Datos')

Leyendo archivo...


In [44]:
#Reemplazar el nombre de la columna "Día:" y "Mes_N" para estandarizar los scripts
df_Mezclas.rename(columns={"Día:":"Dia", "Mes_N":"IdMes"}, inplace=True)

In [45]:
#Eliminar_filas_vacias
numeros_vacios_filtrados = fc.eliminar_filas_vacias(df_Mezclas, 'Número')

In [46]:
#Crear_fecha
df_Mezclas = fc.crear_fecha(df_Mezclas)

In [47]:
#Tomaré solo las columnas de interes del informe de mezclas para el sobrepeso
columnas=['Número','Dia','Fecha','Mes:', 'IdMes', 'Año:', 'Máquina / Equipo:', 'Semana:', 'Turno:', 'Código y Descrip. / Producto:', 'Unidades Producidas (Conformes) :', 'Peso Promedio de la unidad (K):','Gramaje (K):']
df_Mezclas = df_Mezclas[columnas]

In [48]:
#Filtrar_desde_anio
df_Mezclas = fc.filtrar_desde_anio(df_Mezclas, 2024)

In [49]:
#Especificar las columnas en las que deseas aplicar el filtro para eliminar los valores nulos de las columnas de Gramaje, Unidades producidas y Peso promedio
columnas_filtrar_nulos = ['Gramaje (K):', 'Unidades Producidas (Conformes) :', 'Peso Promedio de la unidad (K):']

In [50]:
#Eliminar_valores_nulos
df_Mezclas_nan_ceros, df_Mezclas = fc.filtrar_nulos(df_Mezclas, columnas_filtrar_nulos)

**Modificación de los tipos de datos del archivo de TPM de Mezclas**

In [51]:
#Validar_y_convertir_datos
df_non_conver, df_Mezclas = fc.validar_numericos(df_Mezclas, 'Peso Promedio de la unidad (K):')

In [52]:
#Extraer_codigo
df_Mezclas = fc.extraer_codigo(df_Mezclas)

In [53]:
#Convertir a numerico sino asignar NaN
df_Mezclas['Gramaje (K):'] = pd.to_numeric(df_Mezclas['Gramaje (K):'], errors='coerce')

In [54]:
#Eliminar_codigos_nan
df_Mezclas, df_Mezclas_Codigos_nan = fc.eliminar_codigos_nan(df_Mezclas)

In [55]:
#Calcular_columnas
df_Mezclas = fc.calcular_columnas(df_Mezclas)

**Generación de Novedades**

In [56]:
Sobrepeso_Novedades = 0.05 #5% de Sobrepeso

In [57]:
#Generar_novedades
df_Mezclas_Nov_Sobrepeso = fc.generar_novedades(df_Mezclas, Sobrepeso_Novedades)

In [58]:
#Agregar columna 'Costo/kg'
df_Mezclas['Costo/kg'] = 0.0

**Tabla de Consolidado Mensual**

In [59]:
#Pivote_consolidado
df_Mezclas_Mes = fc.pivote_consolidado(df_Mezclas)

In [60]:
#Tomar datos desde el año 2024 en adelante
df_Mezclas_Mes = fc.filtrar_desde_anio(df_Mezclas_Mes, 2024)

**Creación de la agrupación Mensual del archivo de semielaborados**

In [61]:
df_costo_semi = df_COOISPI_COMB.copy(deep=True)

In [62]:
#Creacion de la agrupacion mensual del archivo semielaborado
df_costo_semi_Mes = fc.pivote_semielaborados(df_costo_semi)

**Generación del nuevo archivo consolidado V2**

In [63]:
df_Mezclas_2 = df_Mezclas.copy(deep=True)

In [64]:
#Traer la columna 'Costo/kg' al dataframe original
df_Mezclas_2 = fc.merge_costo(df_Mezclas_2, df_costo_semi_Mes)

In [65]:
#Eliminar la columna Costo/kg antigua y renombrar la nueva
df_Mezclas_2 = fc.modificar_costo(df_Mezclas_2)

In [66]:
#Definir meta
df_Mezclas_2['Meta'] = df_Mezclas_2['Máquina / Equipo:'].map({'ENFLEX': 0.0056,
                                                              'TRANSPACK 2': 0.0033,
                                                              'INGEPACK': 0.0072,
                                                              'ROVEMA':0.0078,
                                                              'LANIC':0.0055,
                                                              'TOYO 7':0.0065,
                                                              'TOYO':0.0088,
                                                              'ENCAPSULADORA':0.006,
                                                              'INGEPACK 2':0.0068})

In [ ]:
#Calcular columna 'Ahorros/Perdidas'
df_Mezclas_2 = fc.calcular_ahorros_perdidas(df_Mezclas_2)
# df_Mezclas_2.head()

,Número,Dia,Fecha,Mes:,IdMes,Año:,Máquina / Equipo:,Semana:,Turno:,Código y Descrip. / Producto:,...,Peso Promedio de la unidad (K):,Gramaje (K):,Codigo,Real_empaque_calculado,Debe_ser_empaque_calculado,Diferencia_calculado,Sobrepeso_calculado,Costo/kg,Meta,Ahorros_Perdidas
0,74528,2,2024-01-02,Enero,1,2024,ENFLEX,1,1,1014168 Polvo st1 10g sobre ind. Horiz Ecu.,...,0.01010,0.0100,1014168,30.3000,30.00,0.3000,0.010000,55278.0,0.0056,-7369.662960
1,74529,2,2024-01-02,Enero,1,2024,INGEPACK 2,1,1,1057776 Cafe Colcafe Clasico10g25sobres,...,0.01005,0.0100,1057776,195.9750,195.00,0.9750,0.005000,40439.0,0.0068,14265.059445
2,74531,2,2024-01-02,Enero,1,2024,TOYO,1,3,1056678 Cafe COLCAFE clasico 1.5g 48sob 30ple,...,0.00151,0.0015,1056678,160.9056,159.84,1.0656,0.006667,40439.0,0.0088,13881.304658
3,74532,2,2024-01-02,Enero,1,2024,ROVEMA,1,3,1057775 Cafe COLCAFE clasico 10g10sob12tarj,...,0.01009,0.0100,1057775,274.8516,272.40,2.4516,0.009000,40439.0,0.0078,-13337.668623
4,74533,2,2024-01-02,Enero,1,2024,INGEPACK,1,3,1003690 Cafe COLCAFE polvo ST1 Cj25tirilX24X10g,...,0.01007,0.0100,1003690,259.8060,258.00,1.8060,0.007000,55278.0,0.0072,2872.311214


In [68]:
#Eliminar registro con Número = 81187
df_Mezclas_2.drop(df_Mezclas_2.loc[df_Mezclas_2['Número'] == 81187].index, inplace=True)

**Unión de los dataframes en un dataframe totalizado**

In [69]:
df_totalizado_Mes = df_Mezclas_Mes.copy(deep=True)

In [70]:
#Validar que no hay filas repetidas en df_totalizado_Mes
fc.filas_repetidas(df_totalizado_Mes)

No hay filas duplicadas.


In [71]:
#Fusionar los DataFrames en base a 'Año:', 'Codigo' y 'Mes:'
df_totalizado_Mes = fc.unir_dataframes(df_totalizado_Mes, df_costo_semi_Mes)

In [ ]:
#Metodo diferente a los demas calcular_columnas_totalizado ya que sus metas de sobrepeso no son fijas
df_totalizado_Mes = fc.calcular_columnas_totalizado_mezclas(df_totalizado_Mes)
# df_totalizado_Mes.head()

,Año:,Mes:,Codigo,Máquina / Equipo:,Debe_ser_empaque_calculado,Diferencia_calculado,Gramaje (K):,Peso Promedio de la unidad (K):,Real_empaque_calculado,Unidades Producidas (Conformes) :,Ctd.,Impte.ML,Real_empaque,Debe_ser_empaque,Diferencia,Sobrepeso,Meta,Costo/kg,Ahorros_Perdidas
0,2024,Abril,1002807,ENFLEX,2009.05,18.94068,0.025,0.025253,2027.99068,80362.0,1990.000,77354198.0,2.029370,2.00905,0.020320,0.010114,0.0056,38871.456281,-356.107493
1,2024,Abril,1003359,TRANSPACK 2,13581.50,59.07912,0.500,0.502275,13640.57912,27163.0,17446.108,602763406.0,13.643285,13.58150,0.061785,0.004549,0.0033,34550.021472,-588.857157
2,2024,Abril,1003690,INGEPACK,2874.00,28.94400,0.010,0.010101,2902.94400,287400.0,4129.000,170292347.0,2.902945,2.87400,0.028945,0.010071,0.0072,41243.000000,-343.785152
3,2024,Abril,1003690,ROVEMA,1314.00,12.37800,0.010,0.010093,1326.37800,131400.0,4129.000,170292347.0,1.326264,1.31400,0.012264,0.009333,0.0078,41243.000000,-83.871963
4,2024,Abril,1014168,INGEPACK,496.00,6.88900,0.010,0.010130,502.88900,49600.0,505.000,20827715.0,0.502448,0.49600,0.006448,0.013000,0.0072,41243.000000,-120.190285


In [73]:
# Obtener Codigos unicos de df_Mezclas
df_Codigos_Mezclas = fc.eliminar_duplicados_columna(df_Mezclas, 'Codigo')

In [74]:
# Obtener Maquinas únicas de TPM o df_Codigos_Mezclas
df_Maquinas_Mezclas = fc.eliminar_duplicados_columna(df_Mezclas, 'Máquina / Equipo:')

In [75]:
# Obtener Codigos únicos de COOISPI
df_Codigos_Mezclas_COOISPI = fc.eliminar_duplicados_columna(df_COOISPI_COMB, 'Codigo')

**Codigos que están en TPM pero no están en la COOISPI**

In [76]:
#Obtener Codigos que están en TPM (df_Codigos_Mezclas) pero NO en COOISPI(df_Codigos_Mezclas_COOISPI)
df_Mezclas_Codigos_unicos = df_Codigos_Mezclas[~df_Codigos_Mezclas['Codigo'].isin(df_Codigos_Mezclas_COOISPI['Codigo'])]

**Generación de un archivo de fechas Automáticas**

In [77]:
#Generar dataframe con fechas dese 2024-01-01 hasta el dia actual
df_Fechas = fc.generar_fechas("2024-01-01")

In [78]:
# print(df_Mezclas.shape)
# print(df_COOISPI_COMB.shape)
# print(df_Mezclas_Mes.shape)
# print(df_Codigos_Mezclas.shape)
# print(df_Maquinas_Mezclas.shape)
# print(df_totalizado_Mes.shape)
# print(df_Fechas.shape)
# print(df_Mezclas_2.shape)
# print()
# print(df_Mezclas_Nov_Sobrepeso.shape)
# print(df_Mezclas_Codigos_nan.shape)
# print(df_Mezclas_Codigos_unicos.shape)
# print(df_Mezclas_nan_ceros.shape)
# print(df_non_conver.shape)

**Consolidado csv**

In [79]:
#Ruta para consolidado df_Mezclas_2
df_Mezclas_2.to_csv(Ruta_Mezclas + "DB_Mezclas_Out_vf.csv", index=False)

**Archivo de consolidado**

In [80]:
#Diccionario de hojas para el archivo de Consolidado_V2
dataframes_consolidado = {
    "Hoja1": df_Mezclas,
    "Hoja2": df_COOISPI_COMB,
    "Hoja3": df_Mezclas_Mes,
    "Hoja4": df_Codigos_Mezclas,
    "Hoja5": df_Maquinas_Mezclas,
    "Hoja6": df_totalizado_Mes,
    "Hoja7": df_Fechas,
    "Hoja8": df_Mezclas_2,
}
#Actualizar hojas del excel de Consolidado_V2
fc.actualizar_hojas_excel(Ruta_Consolidados, dataframes_consolidado)

Procesando Consolidado...
✅ Las hojas en el Archivo de Consolidado han sido limpiadas y actualizadas correctamente.


**Archivo de novedades: Este archivo compila las diferentes novedades presentes en los archivos de TPM**

In [81]:
#Diccionario de hojas para el archivo de novedades
dataframes_novedades = {
    "Sobrepeso mayor  5%": df_Mezclas_Nov_Sobrepeso,
    "Codigos Nulo TPM": df_Mezclas_Codigos_nan,
    "Codigos unicos en TPM": df_Mezclas_Codigos_unicos,
    "Codigos NA,Vacio, 0": df_Mezclas_nan_ceros,
    "Errores": df_non_conver
}
#Guardar el archivo de Novedades
fc.crear_archivo_novedades(Ruta_Novedades, dataframes_novedades)

Procesando Novedades...
✅ El Archivo de Novedades guardado exitosamente con 5 hojas guardadas correctamente.
